<a href="https://colab.research.google.com/github/Leucocitokiller/Proyecto-Fina-NLP/blob/main/NLP_Jack_Reacher.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 📚 Análisis NLP sobre texto clásico (Don Quijote)
Este notebook realiza preprocesamiento y análisis de texto usando múltiples técnicas de NLP aplicadas sobre el texto de 'Don Quijote' obtenido desde el Proyecto Gutenberg.

In [ ]:
import nltk

In [ ]:
# 📥 1. Cargar texto desde URL
import requests

url = 'https://www.gutenberg.org/files/2000/2000-0.txt'
texto = requests.get(url).text
print(texto[:1000])  # Mostrar los primeros 1000 caracteres


In [ ]:
# 🧠 2. Procesamiento NLP con spaCy
import spacy
from spacy.lang.es.stop_words import STOP_WORDS

# Download the model if it's not installed
!python -m spacy download es_core_news_sm

nlp = spacy.load("es_core_news_sm")

# Increase the max_length limit
nlp.max_length = len(texto)  # Set to the length of your text

doc = nlp(texto)


In [ ]:
# 🧹 3. Lematización, remoción de stopwords y puntuación
tokens_limpios = [token.lemma_.lower() for token in doc
                  if token.is_alpha and token.lemma_.lower() not in STOP_WORDS]
print(tokens_limpios[:30])


In [ ]:
# 🔠 4. POS Tagging (etiquetado gramatical)
for token in doc[:10]:
    print(f"{token.text} -> {token.pos_}")


In [ ]:
# 🧾 5. Named Entity Recognition (NER)
for ent in doc.ents[:10]:
    print(f"{ent.text} -> {ent.label_}")


In [ ]:
# 📊 6. Palabras más frecuentes
from collections import Counter

frecuencia = Counter(tokens_limpios).most_common(10)
for palabra, freq in frecuencia:
    print(f"{palabra}: {freq}")


In [ ]:
# ☁️ 7. WordCloud
from wordcloud import WordCloud
import matplotlib.pyplot as plt

wordcloud = WordCloud(width=800, height=400, background_color='white').generate(' '.join(tokens_limpios))
plt.figure(figsize=(12, 6))
plt.imshow(wordcloud, interpolation='bilinear')
plt.axis("off")
plt.title("Word Cloud del texto")
plt.show()


In [ ]:
# 📛 8. Bigramas más frecuentes
bigrams = zip(tokens_limpios, tokens_limpios[1:])
bigrams_freq = Counter(bigrams).most_common(10)

for par, freq in bigrams_freq:
    print(f"{par}: {freq}")


In [ ]:
import seaborn as sns
import pandas as pd

palabras_df = pd.DataFrame(frecuencia, columns=['palabra', 'frecuencia'])
sns.barplot(data=palabras_df, x='frecuencia', y='palabra')
plt.title("Top 10 palabras más frecuentes")
plt.xlabel("Frecuencia")
plt.ylabel("Palabra")
plt.show()

In [ ]:
ents = [ent.text for ent in doc.ents if ent.label_ in ['PER', 'LOC', 'ORG']]
ent_freq = Counter(ents).most_common(10)
pd.DataFrame(ent_freq, columns=["Entidad", "Frecuencia"]).plot.bar(x='Entidad', y='Frecuencia', legend=False)
plt.title("Entidades nombradas más frecuentes")
plt.ylabel("Frecuencia")
plt.show()

In [ ]:
from collections import Counter
pos_counts = Counter([token.pos_ for token in doc if token.is_alpha])
sns.barplot(x=list(pos_counts.keys()), y=list(pos_counts.values()))
plt.title("Distribución de categorías gramaticales")
plt.xlabel("POS")
plt.ylabel("Cantidad")
plt.show()


In [ ]:
import networkx as nx

G = nx.Graph()
for (w1, w2), freq in bigrams_freq:
    G.add_edge(w1, w2, weight=freq)

plt.figure(figsize=(10,6))
nx.draw_networkx(G, with_labels=True, node_size=1500, font_size=10)
plt.title("Bigramas más frecuentes")
plt.show()




In [ ]:
from textblob import TextBlob

# Assuming 'tokens_limpios' from previous cell contains the processed text
texto_procesado = ' '.join(tokens_limpios)  # Join the tokens into a string

# Análisis de sentimientos por párrafo o línea
sentimientos = []
for frase in texto_procesado.split('\n'):
    blob = TextBlob(frase)
    sentimientos.append((frase, blob.sentiment.polarity))

# Mostrar las frases más positivas y más negativas
sentimientos_ordenados = sorted(sentimientos, key=lambda x: x[1])
print("Frase más negativa:\n", sentimientos_ordenados[0])
print("\nFrase más positiva:\n", sentimientos_ordenados[-1])


In [ ]:

from nltk.sentiment import SentimentIntensityAnalyzer
import nltk
nltk.download('vader_lexicon')

sia = SentimentIntensityAnalyzer()

sentimientos = []
for frase in texto_procesado.split('\n'):
    score = sia.polarity_scores(frase)['compound']
    sentimientos.append((frase, score))

# Frases con sentimiento más marcado
sentimientos_ordenados = sorted(sentimientos, key=lambda x: x[1])
print("Frase más negativa:\n", sentimientos_ordenados[0])
print("\nFrase más positiva:\n", sentimientos_ordenados[-1])


In [ ]:
import matplotlib.pyplot as plt

polaridades = [s[1] for s in sentimientos]
plt.figure(figsize=(12, 4))
plt.plot(polaridades)
plt.title("Evolución del sentimiento a lo largo del texto")
plt.xlabel("Línea del texto")
plt.ylabel("Polaridad (-1 a 1)")
plt.show()
